# Chapter 27: GitLab CI/CD

While Chapters 25 and 26 examined Jenkins and GitHub Actions, this chapter explores GitLab CI/CD—an integral component of GitLab's single-application DevOps platform. Unlike the plugin-dependent architecture of Jenkins or the event-driven model of GitHub Actions, GitLab CI/CD offers a unified experience where source control, CI/CD, container registries, security scanning, and deployment monitoring coexist within the same interface.

GitLab's approach emphasizes convention over configuration through features like Auto DevOps, while providing deep Kubernetes integration for modern cloud-native workflows. Understanding GitLab's YAML syntax, runner architecture, and built-in security capabilities enables teams to leverage a comprehensive DevOps platform without the integration complexity of disparate tools.

## 27.1 GitLab CI Architecture

GitLab CI/CD operates on a distributed architecture comprising the GitLab instance, runners, and executors, designed to scale from single-developer projects to enterprise-wide deployments.

### Core Components

**GitLab Instance:**
The central coordination server providing:
- **Pipeline Configuration**: Parsing and validation of `.gitlab-ci.yml`
- **Job Scheduling**: Queue management and runner selection
- **Artifact Storage**: Built-in artifact repository with retention policies
- **Container Registry**: Integrated Docker registry per project
- **Security Dashboard**: Centralized vulnerability management

**Runners:**
Agents that execute CI/CD jobs, available in two types:
- **Shared Runners**: Provided by GitLab.com (or shared infrastructure) for multiple projects
- **Specific Runners**: Dedicated to individual projects or groups, offering specialized hardware or network access

**Executors:**
The runtime environment where jobs execute:
- **Shell**: Direct execution on the runner host (least isolated)
- **Docker**: Container-based execution (recommended default)
- **Kubernetes**: Dynamic pod creation per job (cloud-native)
- **VirtualBox/Parallels**: VM-based execution for full isolation
- **Docker Machine**: Autoscaling Docker hosts on cloud providers (AWS, GCP, Azure)

### Architecture Diagram

```
GitLab Instance
    │
    ├── Web UI (Pipeline visualization, job logs)
    ├── API (Trigger pipelines, fetch artifacts)
    └── Registry (Container images, packages)
         │
         ▼
    Job Queue (Redis-backed)
         │
         ├──────► GitLab Runner (Docker executor)
         │           └── Docker Container (ephemeral)
         │
         ├──────► GitLab Runner (Kubernetes executor)
         │           └── Pod (dynamic, per-job)
         │
         └──────► GitLab Runner (Shell executor)
                     └── Host process
```

### Runner Installation and Registration

```bash
# Install GitLab Runner on Ubuntu
curl -L "https://packages.gitlab.com/install/repositories/runner/gitlab-runner/script.deb.sh" | sudo bash
sudo apt-get install gitlab-runner

# Register runner with GitLab instance
sudo gitlab-runner register \
  --non-interactive \
  --url "https://gitlab.com/" \
  --registration-token "REGISTRATION_TOKEN" \
  --executor "docker" \
  --docker-image docker:24-dind \
  --description "docker-runner" \
  --tag-list "docker,linux,amd64" \
  --run-untagged="false" \
  --locked="false" \
  --access-level="not_protected"

# Kubernetes Runner (Helm installation)
helm repo add gitlab https://charts.gitlab.io
helm install gitlab-runner gitlab/gitlab-runner \
  --set gitlabUrl=https://gitlab.com/ \
  --set runnerRegistrationToken=REGISTRATION_TOKEN \
  --set rbac.create=true \
  --set runners.executor=kubernetes \
  --set runners.tags={kubernetes}
```

## 27.2 .gitlab-ci.yml Syntax

The `.gitlab-ci.yml` file defines pipeline structure using YAML with GitLab-specific keywords, supporting complex workflows through rules, parallelization, and template inheritance.

### Basic Structure

```yaml
# .gitlab-ci.yml
stages:          # Define pipeline phases
  - build
  - test
  - security
  - deploy

variables:       # Global variables
  DOCKER_DRIVER: overlay2
  DOCKER_TLS_CERTDIR: "/certs"
  NODE_VERSION: "20"

# Global defaults for all jobs
default:
  image: node:20-alpine
  before_script:
    - npm ci --cache .npm --prefer-offline
  cache:
    key:
      files:
        - package-lock.json
    paths:
      - .npm/

build:job:       # Job name (arbitrary, but descriptive)
  stage: build
  script:
    - npm run build
  artifacts:
    paths:
      - dist/
    expire_in: 1 week

test:unit:
  stage: test
  script:
    - npm run test:unit -- --coverage
  coverage: '/All files[^|]*\|[^|]*\s+([\d\.]+)/'
  artifacts:
    reports:
      coverage_report:
        coverage_format: cobertura
        path: coverage/cobertura-coverage.xml

deploy:staging:
  stage: deploy
  script:
    - echo "Deploy to staging"
  environment:
    name: staging
    url: https://staging.example.com
  only:
    - main
```

### Advanced Syntax Features

**Rules (Modern conditional syntax replacing only/except):**
```yaml
deploy:production:
  stage: deploy
  script:
    - helm upgrade --install myapp ./chart
  rules:
    # Deploy on main branch when tests pass
    - if: $CI_COMMIT_BRANCH == "main"
      when: manual
      allow_failure: false
    
    # Deploy on tags automatically
    - if: $CI_COMMIT_TAG
      when: on_success
    
    # Skip on draft MRs
    - if: $CI_MERGE_REQUEST_TITLE =~ /Draft:/
      when: never
    
    # Default: don't run
    - when: never
```

**Parallel and Matrix Jobs:**
```yaml
# Parallel test splitting
test:parallel:
  stage: test
  parallel: 5  # Creates 5 jobs numbered 1/5, 2/5, etc.
  script:
    - npm test -- --shard=$CI_NODE_INDEX/$CI_NODE_TOTAL

# Matrix builds (cross-product)
test:matrix:
  stage: test
  parallel:
    matrix:
      - PROVIDER: [aws, gcp, azure]
        REGION: [us-east, eu-west, asia-pacific]
      - PROVIDER: [onprem]
        REGION: [datacenter1, datacenter2]
  script:
    - echo "Testing $PROVIDER in $REGION"
```

**Needs (DAG - Directed Acyclic Graph):**
```yaml
# Skip sequential stages, create dependencies
build:frontend:
  stage: build
  script: npm run build:frontend

build:backend:
  stage: build
  script: npm run build:backend

test:integration:
  stage: test
  needs: [build:frontend, build:backend]  # Wait only for these, not whole stage
  script: npm run test:integration

deploy:
  stage: deploy
  needs: [test:integration]  # Explicit dependency
  script: echo "Deploying"
```

## 27.3 Pipeline Stages and Jobs

GitLab pipelines organize work into stages executed sequentially, with jobs within stages running in parallel by default.

### Stage Configuration

```yaml
stages:
  - .pre        # Implicit stage (runs first)
  - build
  - test
  - security
  - deploy
  - .post       # Implicit stage (runs last)

# Job in implicit stage
workflow:
  rules:
    - if: $CI_PIPELINE_SOURCE == "merge_request_event"
    - if: $CI_COMMIT_BRANCH == "main"

# Hidden job (template, won't run)
.unit_test_template:
  stage: test
  image: node:20-alpine
  script:
    - npm test
  artifacts:
    reports:
      junit: junit.xml

# Extending templates
test:unit:
  extends: .unit_test_template
  variables:
    TEST_SUITE: unit

test:integration:
  extends: .unit_test_template
  services:
    - postgres:15
  variables:
    TEST_SUITE: integration
```

### Job Control Flow

**Interruptible Jobs (Stop redundant pipelines):**
```yaml
build:
  stage: build
  interruptible: true  # Cancel if newer pipeline starts for same branch
  script:
    - long_running_build.sh
```

**Retry Logic:**
```yaml
test:flaky:
  script:
    - npm test
  retry:
    max: 2
    when:
      - runner_system_failure
      - stuck_or_timeout_failure
      - script_failure
```

**Timeout Control:**
```yaml
deploy:
  timeout: 30m  # Job-specific timeout
  script:
    - helm upgrade --wait --timeout 20m myapp ./chart
```

### Resource Groups (Prevent Concurrent Deployments)

```yaml
deploy:production:
  stage: deploy
  resource_group: production  # Mutual exclusion
  script:
    - ./deploy.sh
  environment:
    name: production
```

## 27.4 Docker Integration

GitLab CI/CD provides first-class Docker support through the Docker executor and built-in registry integration.

### Docker-in-Docker (DinD)

```yaml
build:image:
  stage: build
  image: docker:24
  services:
    - docker:24-dind  # Sidecar container with Docker daemon
  variables:
    DOCKER_TLS_CERTDIR: "/certs"
    DOCKER_DRIVER: overlay2
    IMAGE_TAG: $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
  before_script:
    - docker login -u $CI_REGISTRY_USER -p $CI_REGISTRY_PASSWORD $CI_REGISTRY
  script:
    - docker build --pull -t $IMAGE_TAG .
    - docker push $IMAGE_TAG
    - docker tag $IMAGE_TAG $CI_REGISTRY_IMAGE:latest
    - docker push $CI_REGISTRY_IMAGE:latest
```

### Kaniko (Rootless Building)

Avoid privileged containers for enhanced security:

```yaml
build:kaniko:
  stage: build
  image:
    name: gcr.io/kaniko-project/executor:debug
    entrypoint: [""]
  script:
    - /kaniko/executor
      --context "${CI_PROJECT_DIR}"
      --dockerfile "${CI_PROJECT_DIR}/Dockerfile"
      --destination "${CI_REGISTRY_IMAGE}:${CI_COMMIT_SHA}"
      --destination "${CI_REGISTRY_IMAGE}:latest"
      --cache=true
      --cache-ttl=24h
```

### Multi-Platform Builds

```yaml
build:multiarch:
  stage: build
  image: docker:24
  services:
    - docker:24-dind
  variables:
    DOCKER_BUILDKIT: 1
  before_script:
    - docker login -u $CI_REGISTRY_USER -p $CI_REGISTRY_PASSWORD $CI_REGISTRY
    - docker buildx create --use
  script:
    - docker buildx build 
      --platform linux/amd64,linux/arm64 
      -t $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA 
      --push .
```

## 27.5 Kubernetes Integration

GitLab offers deep Kubernetes integration through the Kubernetes executor and cluster management features.

### Kubernetes Executor

```yaml
# Config.toml for Kubernetes executor
[[runners]]
  executor = "kubernetes"
  [runners.kubernetes]
    namespace = "gitlab-runner"
    image = "ubuntu:22.04"
    [runners.kubernetes.volumes.empty_dir]
      name = "cache"
      mount_path = "/cache"
      medium = "Memory"
```

**Pipeline using Kubernetes executor:**
```yaml
deploy:k8s:
  stage: deploy
  image: bitnami/kubectl:latest
  script:
    - kubectl config use-context $KUBE_CONTEXT
    - kubectl set image deployment/myapp 
        app=$CI_REGISTRY_IMAGE:$CI_COMMIT_SHA 
        --namespace production
    - kubectl rollout status deployment/myapp --namespace production
```

### GitLab Agent for Kubernetes

For secure cluster connections without exposing APIs:

```yaml
# .gitlab/agents/k8s-agent/config.yaml
ci_access:
  projects:
    - id: group/project
```

**Pipeline using Agent:**
```yaml
deploy:agent:
  stage: deploy
  image: bitnami/kubectl:latest
  script:
    - kubectl config get-contexts  # Lists agent-registered contexts
    - kubectl --context path/to/agent apply -f deployment.yaml
```

## 27.6 GitLab Registry

GitLab includes a container registry per project, tightly integrated with CI/CD authentication.

### Registry Operations

```yaml
variables:
  IMAGE_NAME: $CI_REGISTRY_IMAGE/myapp

build:
  stage: build
  script:
    # Login automatically provided via CI_JOB_TOKEN
    - docker login -u $CI_REGISTRY_USER -p $CI_JOB_TOKEN $CI_REGISTRY
    
    # Build with cache
    - docker build 
      --cache-from $IMAGE_NAME:latest 
      -t $IMAGE_NAME:$CI_COMMIT_SHA 
      -t $IMAGE_NAME:latest 
      .
    
    - docker push $IMAGE_NAME:$CI_COMMIT_SHA
    - docker push $IMAGE_NAME:latest
    
    # Write digest for downstream jobs
    - echo $IMAGE_NAME:$CI_COMMIT_SHA > image.txt
  artifacts:
    paths:
      - image.txt
```

### Registry Cleanup Policies

Automatically remove old images to save storage:

```yaml
# Project Settings > Packages & Registries > Cleanup policies
variables:
  # Keep only last 10 tags per image
  REGISTRY_RETENTION_DAYS: "30"
```

**Pipeline-based cleanup:**
```yaml
cleanup:registry:
  stage: cleanup
  image: registry.gitlab.com/gitlab-org/gitlab-runner/gitlab-runner-helper:x86_64-latest
  script:
    - 'curl --request DELETE --header "PRIVATE-TOKEN: $CI_JOB_TOKEN" 
        "${CI_API_V4_URL}/projects/${CI_PROJECT_ID}/registry/repositories"'
  when: manual
```

## 27.7 Variables and Secrets

GitLab provides hierarchical variable management from instance to project levels, with protected and masked variable support.

### Variable Types

```yaml
# In .gitlab-ci.yml
job:
  variables:
    DEPLOY_ENV: "staging"  # Job-level
    
  script:
    - echo $CI_COMMIT_SHA      # Predefined GitLab variable
    - echo $DEPLOY_ENV         # Custom variable
    - echo $DATABASE_PASSWORD  # Project Settings > CI/CD > Variables
```

**Project Variables (UI Configuration):**
- **Protected**: Only available on protected branches
- **Masked**: Hidden in job logs (replaced with `[MASKED]`)
- **Expanded**: Support variable referencing (`$CI_REGISTRY_IMAGE`)

### CI/CD Variables Hierarchy

1. Instance variables (Admin area)
2. Group variables
3. Project variables
4. Pipeline variables (triggered via API)
5. Job variables
6. `.gitlab-ci.yml` variables

### File Variables

Mount secrets as files:

```yaml
deploy:
  script:
    - kubectl apply -f deployment.yaml
  variables:
    KUBECONFIG: $KUBE_CONFIG_DATA  # Variable type: File
```

## 27.8 Artifacts and Caching

GitLab distinguishes between artifacts (passed between stages/jobs) and cache (persisted between pipelines).

### Artifacts

Pass build outputs downstream:

```yaml
build:
  script:
    - npm run build
  artifacts:
    name: "$CI_JOB_NAME-$CI_COMMIT_REF_NAME"
    paths:
      - dist/
      - node_modules/.package-lock.json
    reports:
      junit: test-results.xml
      coverage_report:
        coverage_format: cobertura
        path: coverage/cobertura-coverage.xml
    expire_in: 1 week
    when: on_success  # or always/never
```

**Dependency Download:**
```yaml
deploy:
  dependencies:
    - build  # Download artifacts from specific job
  script:
    - ls dist/  # Artifacts available here
```

### Cache

Persist between pipeline runs:

```yaml
default:
  cache:
    key:
      files:
        - package-lock.json  # Cache key based on lock file
    paths:
      - node_modules/
      - .npm/
    policy: pull-push  # Default: download at start, upload at end

# Don't update cache for scheduled pipelines (read-only)
cleanup:
  cache:
    policy: pull  # Read-only cache
  script:
    - npm run cleanup
  rules:
    - if: $CI_PIPELINE_SOURCE == "schedule"
```

**Cache vs Artifacts:**

| Feature | Artifacts | Cache |
|---------|-----------|-------|
| **Purpose** | Pass between jobs/stages | Speed up subsequent pipelines |
| **Persistence** | 1 week default (configurable) | Until overwritten or cleared |
| **Scope** | Pipeline-local | Shared across pipelines |
| **Use Case** | Compiled binaries, test reports | Dependencies, package managers |

## 27.9 Environments and Deployments

GitLab's environment model provides deployment tracking, rollback capabilities, and monitoring integration.

### Environment Configuration

```yaml
deploy:staging:
  stage: deploy
  script:
    - helm upgrade --install myapp ./chart --namespace staging
  environment:
    name: staging
    url: https://staging.example.com
    on_stop: stop_staging  # Associated cleanup job

stop_staging:
  stage: deploy
  script:
    - helm uninstall myapp --namespace staging
  environment:
    name: staging
    action: stop
  when: manual  # Only run when explicitly triggered
```

### Deployment Strategies

**Basic Deployment:**
```yaml
deploy:
  environment:
    name: production
    url: https://api.example.com
  script:
    - ./deploy.sh
```

**Canary Deployment (Progressive rollout):**
```yaml
deploy:canary:
  stage: deploy
  script:
    - kubectl apply -f canary-deployment.yaml
    - sleep 300  # Wait 5 minutes, monitor metrics
    - ./promote-canary.sh
  environment:
    name: production-canary
```

### Review Apps (Dynamic Environments)

Create temporary environments for Merge Requests:

```yaml
review:
  stage: deploy
  script:
    - helm install 
        --set image.tag=$CI_COMMIT_SHA 
        --namespace review 
        --create-namespace 
        review-$CI_MERGE_REQUEST_IID ./chart
    - echo "URL=https://review-$CI_MERGE_REQUEST_IID.example.com" >> deploy.env
  environment:
    name: review/$CI_MERGE_REQUEST_SOURCE_BRANCH_NAME
    url: https://review-$CI_MERGE_REQUEST_IID.example.com
    on_stop: stop_review
    auto_stop_in: 1 week  # Auto-cleanup
  artifacts:
    reports:
      dotenv: deploy.env  # Pass URL to GitLab UI
  only:
    - merge_requests

stop_review:
  stage: deploy
  script:
    - helm uninstall review-$CI_MERGE_REQUEST_IID --namespace review
  environment:
    name: review/$CI_MERGE_REQUEST_SOURCE_BRANCH_NAME
    action: stop
  when: manual
```

## 27.10 Best Practices

### Security Scanning Integration

GitLab provides built-in security scanners (SAST, DAST, Dependency Scanning, Container Scanning) without external tools:

```yaml
include:
  - template: Security/SAST.gitlab-ci.yml
  - template: Security/Dependency-Scanning.gitlab-ci.yml
  - template: Security/Container-Scanning.gitlab-ci.yml
  - template: Security/DAST.gitlab-ci.yml

# Customize scanners
variables:
  SAST_EXCLUDED_PATHS: "tests/, spec/"
  DAST_WEBSITE: "https://staging.example.com"
```

### Merge Trains

Ensure serialized merges to main, testing the combined result before merging:

```yaml
# Project Settings > General > Merge requests > Enable Merge Trains

merge_train:
  stage: deploy
  script:
    - echo "This runs in merge train context"
  rules:
    - if: $CI_MERGE_REQUEST_EVENT_TYPE == "merge_train"
```

### Pipeline Efficiency

**Workflow Rules (Avoid unnecessary pipelines):**
```yaml
workflow:
  rules:
    # Run for merge requests
    - if: $CI_MERGE_REQUEST_IID
    # Run for tags
    - if: $CI_COMMIT_TAG
    # Run for main branch
    - if: $CI_COMMIT_BRANCH == "main"
    # Don't run for other branches (feature branches only via MR)
    - when: never
```

**Parent-Child Pipelines:**
Split complex pipelines for better organization:

```yaml
# .gitlab-ci.yml
trigger:child:
  trigger:
    include:
      - local: .gitlab/ci/build.gitlab-ci.yml
      - local: .gitlab/ci/test.gitlab-ci.yml
    strategy: depend  # Wait for child completion
```

### Auto DevOps

Zero-configuration CI/CD for standard projects:

```yaml
# Simply include the template
include:
  - template: Auto-DevOps.gitlab-ci.yml

variables:
  AUTO_DEVOPS_DOMAIN: example.com
  POSTGRES_ENABLED: "false"  # Disable if not needed
```

---

## Chapter Summary and Preview

In this chapter, we explored GitLab CI/CD as a comprehensive DevOps platform integrating version control, CI/CD, container registries, and security scanning within a unified interface. We examined the runner architecture supporting multiple executors from Docker to Kubernetes, and the `.gitlab-ci.yml` syntax enabling complex workflows through rules-based conditionals, parallel matrix builds, and directed acyclic graph dependencies. The integrated Container Registry provides seamless authentication via CI job tokens, while Kubernetes integration through agents and executors enables cloud-native deployment patterns. We analyzed environment management supporting review apps for merge requests, canary deployments, and automated cleanup, alongside deployment boards providing visual tracking of releases across environments. The distinction between artifacts (inter-job data transfer) and caching (inter-pipeline acceleration) optimizes storage and performance. Built-in security scanning templates eliminate external tool integration complexity, while merge trains ensure mainline stability through serialized, pre-tested merges. Best practices including workflow rules prevent unnecessary pipeline execution, and parent-child pipelines manage complexity in monorepos.

**Key Takeaways:**
- Leverage GitLab's integrated Container Registry with automatic CI_JOB_TOKEN authentication to eliminate credential management overhead, using cleanup policies to control storage costs
- Implement merge trains for high-velocity teams to prevent broken mainlines, ensuring each merge is tested against the target branch state before integration occurs
- Use the DAG (needs keyword) to optimize pipeline duration by bypassing sequential stage execution when jobs have no interdependencies, reducing wall-clock time significantly
- Distinguish clearly between artifacts (passed between jobs, expire quickly) and cache (persisted between pipelines, optimize for dependency storage) to optimize storage costs and build performance
- Enable Auto DevOps for standardized projects to bootstrap CI/CD immediately, then customize progressively by overriding specific job templates rather than building from scratch
- Utilize review apps with auto-stop-in configuration to provide ephemeral testing environments for merge requests while preventing resource sprawl through automatic cleanup

**Next Chapter Preview:**
Chapter 28: Other CI/CD Tools examines alternative platforms including CircleCI, Azure DevOps Pipelines, AWS CodePipeline, Google Cloud Build, Tekton, and Concourse CI. We will analyze cloud-specific integrations, pricing models, and unique architectural patterns such as CircleCI's orbs for reusable configuration, Azure's tight integration with ARM templates and Entra ID, and Tekton's Kubernetes-native CRD-based approach. This chapter provides decision frameworks for selecting appropriate CI/CD platforms based on existing cloud infrastructure, team size, compliance requirements, and specific workflow patterns, completing our comprehensive survey of modern continuous integration and delivery technologies.